In [1]:
import warnings
warnings.filterwarnings('ignore')
import torch
from pathlib import Path
from PIL import Image
import pandas as pd
import cv2
import numpy as np

In [3]:
# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or 'yolov5m', 'yolov5l', 'yolov5x'

# Define the folder containing the images
image_folder = 'D://Study//Project//Major project//Bio analysis//four_quad_image'
output_folder = 'D://Study//Project//Major project//Bio analysis//output_four_quad_image'

# Ensure the output folder exists
Path(output_folder).mkdir(parents=True, exist_ok=True)

# Get a list of image files in the folder
image_files = list(Path(image_folder).glob('.jpg')) + list(Path(image_folder).glob('.png'))

# Initialize an empty list to store detection results
results_list = []

# Create a function to detect objects in an image and return the results
def detect_objects(image_path):
    # Load the image
    img = Image.open(image_path)
    
    # Perform object detection
    results = model(img)
    
    # Parse results
    detected_objects = results.pandas().xyxy[0]
    
    return detected_objects, results

# Iterate over each image file and detect objects
for image_file in image_files:
    print(f"Processing {image_file.name}...")
    objects, results = detect_objects(image_file)
    
    # Append the results to the list with the image name
    for index, obj in objects.iterrows():
        width = obj['xmax'] - obj['xmin']
        height = obj['ymax'] - obj['ymin']
        area = width * height
        results_list.append({
            'image': image_file.name,
            'xmin': obj['xmin'],
            'ymin': obj['ymin'],
            'xmax': obj['xmax'],
            'ymax': obj['ymax'],
            'confidence': obj['confidence'],
            'class': obj['class'],
            'name': obj['name'],
            'area': area
        })
    
    # Load the image using OpenCV
    img_cv2 = cv2.imread(str(image_file))
    
    # Draw bounding boxes and labels on the image
    for index, obj in objects.iterrows():
        xmin, ymin, xmax, ymax = int(obj['xmin']), int(obj['ymin']), int(obj['xmax']), int(obj['ymax'])
        label = f"{obj['name']} {obj['confidence']:.2f}"
        cv2.rectangle(img_cv2, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(img_cv2, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Save the annotated image with the desired name
    result_img_path = Path(output_folder) / f"yolo_{image_file.name}"
    cv2.imwrite(str(result_img_path), img_cv2)

# Create a DataFrame from the results list
df = pd.DataFrame(results_list)

# Save the DataFrame to an Excel file
output_excel_file = Path(output_folder) / 'detected_objects_with_area.xlsx'
df.to_excel(output_excel_file, index=False)

print(f"Results saved to {output_excel_file}")
print(f"Annotated images saved in {output_folder}")

Using cache found in C:\Users\nishi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-9 Python-3.12.1 torch-2.2.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Results saved to D:\Study\Project\Major project\Bio analysis\output_four_quad_image\detected_objects_with_area.xlsx
Annotated images saved in D://Study//Project//Major project//Bio analysis//output_four_quad_image
